<a href="https://colab.research.google.com/github/TBab0/duth_cv4/blob/main/1sratch%20mod6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import datetime

In [2]:
drive.mount('/content/drive')
local_zip = '/content/drive/MyDrive/Colab Notebooks/imagedb_btsd.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

train_dir = '/content/imagedb'
test_dir = '/content/imagedb_test'

Mounted at /content/drive


In [3]:
ts = 32
# train_datagen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input, validation_split=0.2)
train_datagen  = ImageDataGenerator(rescale=1./255,
                                    validation_split=0.2)

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=128,
                                                    class_mode='categorical',
                                                    color_mode='grayscale',
                                                    target_size=(ts,ts),
                                                    subset='training', seed=1)      
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  train_datagen.flow_from_directory(train_dir,
                                                        batch_size=128,
                                                        class_mode='categorical',
                                                         color_mode='grayscale',
                                                         target_size=(ts,ts),
                                                        subset='validation', seed=1)

Found 2457 images belonging to 34 classes.
Found 599 images belonging to 34 classes.


In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (5, 5), strides=(1, 1), activation='relu', input_shape=(ts, ts, 1)),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(34, activation='softmax')
])
model.compile(optimizer = tf.keras.optimizers.Adam(), loss = 'categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 100)               1254500   
                                                                 
 dense_1 (Dense)             (None, 34)                3434      
                                                                 
Total params: 1,259,598
Trainable params: 1,259,598
Non-trainable params: 0
______________________________________________

In [5]:
callbacks = []

save_best_callback = tf.keras.callbacks.ModelCheckpoint(f'best_weights.hdf5', save_best_only=True, verbose=1)
callbacks.append(save_best_callback)

early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=1)
callbacks.append(early_stop_callback)

history = model.fit(train_generator, validation_data=validation_generator, epochs=30, verbose=1, callbacks=callbacks)

Epoch 1/30
20/20 [==============================] - ETA: 0s - loss: 2.7164 - accuracy: 0.3573
Epoch 1: val_loss improved from inf to 1.93031, saving model to best_weights.hdf5
20/20 [==============================] - 10s 61ms/step - loss: 2.7164 - accuracy: 0.3573 - val_loss: 1.9303 - val_accuracy: 0.5676
Epoch 2/30
19/20 [===========================>..] - ETA: 0s - loss: 1.5343 - accuracy: 0.6702
Epoch 2: val_loss improved from 1.93031 to 1.18066, saving model to best_weights.hdf5
20/20 [==============================] - 1s 51ms/step - loss: 1.5250 - accuracy: 0.6720 - val_loss: 1.1807 - val_accuracy: 0.7212
Epoch 3/30
19/20 [===========================>..] - ETA: 0s - loss: 0.9010 - accuracy: 0.8055
Epoch 3: val_loss improved from 1.18066 to 0.74294, saving model to best_weights.hdf5
20/20 [==============================] - 1s 52ms/step - loss: 0.8926 - accuracy: 0.8087 - val_loss: 0.7429 - val_accuracy: 0.8397
Epoch 4/30
19/20 [===========================>..] - ETA: 0s - loss: 0.528

In [6]:
test_datagen  = ImageDataGenerator(rescale=1./255)
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                         batch_size=32,
                                                         class_mode  = 'categorical',
                                                         color_mode='grayscale',
                                                         target_size=(ts,ts)) 
loss, acc = model.evaluate(test_generator)
print(acc)

Found 2149 images belonging to 34 classes.
68/68 [==============================] - 1s 12ms/step - loss: 0.3047 - accuracy: 0.9418
0.9418334364891052
